# Delay Calculation Validation Experiments

- Note that you'll have to be on an updated branch ... (to be pushed pending exising PRs)
     - There is way to much going on in the current PRs, don't want to add complexity just yet

In [22]:
import os, os.path as op
import numpy as np
import xarray as xr
from datetime import datetime
from RAiDER.llreader import BoundingBox
from RAiDER.losreader import Zenith, Conventional, Raytracing
from RAiDER.delay import tropo_delay
%pdb off

# change this to the directory with the notebook
WD = op.join(op.expanduser('~'), 'Software_InSAR','RAiDER-docs_git','notebooks', 'Synthetic_Test')
os.chdir(WD)
print ('Working directory (WD):', WD)

Automatic pdb calling has been turned OFF
Working directory (WD): /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/Synthetic_Test


### helper functions

update_wm is the one to edit

In [25]:
def update_wm(wm_file):
    """ Edit the values in a weather model file 

    write it out with extension _SYNTH
    Return the file name for input into tropo_delay

    wet_total/hydro_total used for zenith/proj
    wet/hydro used for ray
    """

    with xr.open_dataset(wm_file) as ds:
        # edit t, p, e, wet, hydro, wet_total, hydro_total
        ds['wet_total']   = ('z y x'.split(), np.zeros_like(ds['wet_total']))
        ds['hydro_total'] = ('z y x'.split(), np.zeros_like(ds['hydro_total']))

    dst = f'{op.splitext(wm_file)[0]}_SYNTH.nc'
    ds.to_netcdf(dst)
    print ('Wrote synthetic weather model file to:', dst)
    return dst


In [26]:
def compare_golden(wd, ext='ray'):
    """ Quick check if the synth result is different than real weather wm """
    ds_orig = xr.open_dataset(f'{wd}/golden_data/GMAO_tropo_20181113T230000_{ext}.nc')
    da_dry0 = ds_orig['hydro']
    ds_new  = xr.open_dataset(f'{wd}/Synthetic_tropo_20181113T110000_{ext}.nc')
    da_dry1 = ds_new['hydro']
    equal = np.allclose(da_dry0.data, da_dry1.data, equal_nan=True)
    print ('Orig == synthetic?', equal)
    return

### Get/Set defaults
- Mimic test_scenario_3.py

In [27]:
dt  = datetime(2018, 11, 13, 23)
dts = dt.strftime('%Y_%m_%d_T%H_%M_%S')

height_levels = np.array([0., 100., 500., 1000.])
aoi = BoundingBox([36.8, 36.85, -76.15, -76.05])
aoi.add_buffer(buffer=1.)

# get redownload (with change to name) with !eof -m S1A -d {dts.replace('_', '')}
orbit = f'{WD}/S1A_OPER_AUX_POEORB_OPOD_20181203T120749_V20181112T225942_20181114T005942.EOF'

In [29]:
# original weather model file 
# can be redownloaded with !raider.py raider_example_3.yaml
wm0 = f'{WD}/weather_files/GMAO_{dts}_36N_39N_78W_75W.nc'
wm  = update_wm(wm0)

Wrote synthetic weather model file to: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/Synthetic_Test/weather_files/GMAO_2018_11_13_T23_00_00_36N_39N_78W_75W_SYNTH.nc


In [30]:
# los = Zenith()
# ext = 'zenith'
# los = Conventional(orbit)
# ext = 'proj'

los = Raytracing(orbit, time=dt)
ext = 'ray'

### Compute Delays

In [31]:
ds  = tropo_delay(dt, wm, aoi, los, height_levels, cube_spacing_m=5000)[0]

Output SNWE: [36.75, 36.85, -76.15, -76.0]
Output cube spacing: 0.05
Processing slice 1 / 4: 0.0
Processing slice 2 / 4: 100.0
Processing slice 3 / 4: 500.0
Processing slice 4 / 4: 1000.0


In [32]:
out = f'{WD}/Synthetic_tropo_20181113T110000_{ext}.nc'
ds.to_netcdf(out, mode="w")
print ('Wrote:', out)

Wrote: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/Synthetic_Test/Synthetic_tropo_20181113T110000_ray.nc


### Check against analytical solutions

In [34]:
compare_golden(WD)

Orig == synthetic? True
